In [ ]:
from utils import *
import matplotlib.pyplot as plt

img_path = "../img/Outline/"
# Get the folder information
info = load_folder_information(img_path)
# Generate masks
origin = load_image(os.path.join(img_path, info["origin"]))

print(info)
plt.imshow(origin)

In [ ]:
mask_num = len(info["mask_prefix"])
masks = []
for i in range(mask_num):
    mask = np.zeros_like(origin)
    masked_column = origin.shape[1] // (mask_num + 1) * (i + 1)
    mask[:, masked_column:] = 1
    masks.append(mask)

# for line in range(masks[0].shape[0]):
#     for column in range(masks[0].shape[1]):
#         print(masks[0][line][column][0], end=" ")
#     print()
plt.imshow(masks[6]*255)

In [ ]:
# Image files
img_names = os.listdir(img_path)
# MSE
logits = []
for i in range(mask_num):
    mask = masks[i]
    # for line in range(mask.shape[0]):
    #     for column in range(mask.shape[1]):
    #         print(mask[line][column][0], end=" ")
    #     print()
    mask_prefix = info["mask_prefix"][i]
    imgs = [
        load_image(os.path.join(img_path, img_name))
        for img_name in img_names
        if mask_prefix in img_name
    ]
    mse = [image_MSE(origin, img, mask) for img in imgs]
    logits.append(np.mean(mse))
    # for img in imgs:
    #     plt.imshow(img)
    #     plt.show()

print(logits)

In [ ]:
hpd_path = "/nas/Public/data/HPDv2/test/"
save_path = "/home/lyy/Repos/FACE-image/data/"

import os

file_list = []

if not os.path.exists(save_path):
    os.mkdir(save_path)

for file in os.listdir(hpd_path):
    os.symlink(os.path.join(hpd_path, file), os.path.join(save_path, f"image-{int(file.split('.')[0])}." + file.split(".")[-1] if file.split(".")[-1] != "" else "jpg"))

## results analysis

In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from pygam import LinearGAM

SPLIT = 64
WIDTH = 512
MODEL_NUMBER = 9

hpd_path = "/nas/Public/data/HPDv2/test/"
generate_path = "/nas/Public/experiment_result/FACE-image/"
mapping_path = "/nas/Public/data/HPDv2/test.json"

In [ ]:
mapping = json.load(open(mapping_path))
files = os.listdir(generate_path)

groups = []

for item in mapping:
    group_complete = True
    if len(item["image_path"]) != MODEL_NUMBER:
        # cannot determine which models are used
        continue
    for path in item["image_path"]:
        # iterate image numbers
        number = int(path.split('.')[0])
        single_complete = True
        for i in range(SPLIT-1):
            # check each mask re-generation
            if (f"image-{number}_mask-{i}_00001_.png") not in files:
                single_complete = False
                break
        if not single_complete:
            group_complete = False
            break
    if group_complete:
        groups.append(item["image_path"])

In [ ]:
print(len(groups))
print(groups[0])

In [ ]:
# missing_list = []
# for i in range(1, 3701):
#     for j in range(SPLIT-1):
#         if (f"image-{i}_mask-{j}_00001_.png") not in files:
#             if i not in missing_list:
#                 missing_list.append(i)

### logits calc

In [ ]:
# logits = {}

# for i in range(MODEL_NUMBER):
#     logits[i] = []

# for group in tqdm(groups):
#     for i, file in enumerate(group):
#         subject_img = cv2.imread(os.path.join(generate_path, f"image-{int(file.split('.')[0])}.jpg_00001_.png"), cv2.IMREAD_COLOR)[:,:,::-1]
#         idx = int(file.split('.')[0])
#         # plt.imshow(subject_img)
#         # plt.show()
#         # print(idx)
#         sequence = []
#         # each mask strip
#         for mask_idx in range(SPLIT - 1):
#             generated_img = cv2.imread(os.path.join(generate_path, f"image-{idx}_mask-{mask_idx}_00001_.png"), cv2.IMREAD_COLOR)[:,:,::-1]
#             # one strip
#             columns = WIDTH // SPLIT * (mask_idx + 1)
#             token = np.array(generated_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)
#             ground_truth = np.array(subject_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)

#             mse = ((token - ground_truth) ** 2).mean()
#             sequence.append(mse)
#         logits[i].append(sequence)

# with open("./logits.json", "w") as f:
#     json.dump(logits, f)

In [ ]:
with open("./logits.json", "r") as f:
    logits = json.load(f)

### ranking clac

In [ ]:
rank_lists = {}
rank_avgs = {}
rank_var = {}
for i in range(MODEL_NUMBER):
    rank_lists[i] = []
for item in mapping:
    if len(item["rank"]) != MODEL_NUMBER:
        # cannot determine which models are used
        continue
    for i, rank in enumerate(item["rank"]):
        rank_lists[i].append(rank)

for i in range(MODEL_NUMBER):
    rank_avgs[i] = np.mean(rank_lists[i])
    rank_var[i] = np.var(rank_lists[i])

print(rank_avgs)
print(rank_var)

### plot

In [ ]:
avg_logits = {}
fig_size = (10, 5)
for i, logit_list in logits.items():
    # print(len(logit_list[0]))
    # break
    avg_logits[i] = np.mean(logit_list, axis=0)
    spectra = []
    for line in logit_list:
        spectrum = np.abs(np.fft.fft(line))
        spectra.append(spectrum)

    plt.figure(0, fig_size)
    plt.plot(np.linspace(0, 0.5, len(avg_logits[i])), avg_logits[i], label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}")
    plt.legend()


    mean_spectrum = np.mean(spectra, axis=0)

    mean_spectrum = mean_spectrum[1:]

    plt.figure(1, fig_size)
    plt.plot(np.linspace(0, len(mean_spectrum), len(mean_spectrum)), mean_spectrum, label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}")
    plt.legend()

    X = np.linspace(0, 0.5, len(mean_spectrum))
    gam = LinearGAM().fit(X, mean_spectrum)
    XX = gam.generate_X_grid(term=0, n=len(spectra))
    pdep, confi = gam.partial_dependence(term=0, X=XX, width=0.95)

    # XX =       XX[10:]
    # pdep =   pdep[10:]
    # confi = confi[10:]

    plt.figure(2, fig_size)
    plt.plot(
        XX[:, 0],
        np.abs(pdep),
        label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}",
    )
    # plt.fill_between(
    #     XX[:, 0],
    #     np.where(
    #         confi[:, 0] * confi[:, 1] > 0,
    #         np.minimum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
    #         0
    #     ),
    #     np.maximum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
    #     alpha=0.25
    # )
    plt.legend()
plt.show()

### score